In [1]:
import sys
sys.path.insert(0,r"C:\Users\pc\Documents")
import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
## Lectura de datos     
base_costa = pd.read_csv(r'C:\Users\pc\Downloads\BASE_COSTA.txt', sep=r'¡', engine='python')
base_costa.head()

## Eliminación de variables: 
base_costa2 = base_costa[['CUCONUSE', 'SESUSERV', 'SERVDESC', 'SESUSUSC', 'SESUFEIN','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'SUSCNITC', 
        #'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','V1', 'V2', 'V3', 'V4', 'V5', 'V6',#'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 
        'TIPIFICACION_CLIENTE', 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO',
       #'VECTOR_CUALITATIVO_CONTRATO',#'BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION','RANGO_EDAD', 'GENERO','PROM_SUSC', 'CANAL_ENVIO'
       'SEGMENTACION', 'REGIONAL','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]

base_costa3 = base_costa2.dropna()

In [6]:
## Group by por nit para calificacion
b4 = base_costa3[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
group_nit = b4.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()


## Join de la nueva calificacion
base_costa4 = pd.merge(base_costa3, group_nit, how='left', on=['SUSCNITC'])

## Prueba promedio
b5 = base_costa4[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO_x','CALIFICACION_VECTOR_SERVICIO_y']]

### Creacion de la etiqueta de clasificacion
base_costa4['y'] = pd.cut(x=base_costa4['CALIFICACION_VECTOR_SERVICIO_y'],
                            bins=[-1,50,76,100],
                            labels=['No pago','Pago inoportuno','Pago'])


In [15]:
dataTypeSeries = base_costa4.dtypes
dataTypeSeries

cat = base_costa4.select_dtypes(include=['object','category'])
cuanti = base_costa4.select_dtypes(include=['int64','float64'])
  

le = LabelEncoder()
cat = cat.apply(lambda col: le.fit_transform(col))
cat.head(10)

base_costa5=cuanti.join(cat)
print(base_costa5.dtypes)

base_costa5.to_csv(r'C:\Users\pc\Desktop\base_sur_process.csv')
#base_costa5 = pd.read_csv(r'C:\Users\pc\Desktop\base_sur_process.csv', sep=',', engine='python')


X = base_costa5[[
        #'CUCONUSE', 
        'SESUSERV', 'SERVDESC', 'SESUSUSC','DEPADESC', 'SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 
        #'SESUFEIN','SUSCNITC', 'VECTOR','SESUCATE', 'SESUSUCA','SESULOCA','SESUSAAN','RANGO_EDAD', 'GENERO', 
        #'V1', 'V2', 'V3', 'V4', 'V5', 'V6','V7', 'V8', 'V9', 'V10', 'V11', 'V12','REGIONAL','PROM_SUSC', 'CANAL_ENVIO'     
        # 'SUMA','CANTIDAD','CALIFICACION_VECTOR_SERVICIO_x', 'CALIFICACION_VECTOR_SERVICIO_y',
        #'VECTOR_CUALITATIVO_CONTRATO','BANCO_1', 'DIA_PAGO_1', 'DIA_PAGO_2','DIA_PAGO_3', 'FECHA_SUSPENSION', 
       'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO']]

y = base_costa5['y']


CUCONUSE                            int64
SESUSERV                            int64
SESUSUSC                            int64
SESUCUSA                            int64
SESUCICL                            int64
SUMA                                int64
CANTIDAD                            int64
CALIFICACION_VECTOR_SERVICIO_x      int64
ESTRATO_AGRUPADO                  float64
ANTIGUEDAD_DIAS                     int64
CALIFICACION_VECTOR_SERVICIO_y    float64
SERVDESC                            int32
SESUFEIN                            int32
DEPADESC                            int32
SESUSAPE                            int32
SUSCNITC                            int32
TIPIFICACION_CLIENTE                int32
SEGMENTACION                        int32
REGIONAL                            int32
DEPARTAMENTO_AGR                    int32
CANAL_PAGO                          int32
y                                   int32
dtype: object


In [16]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=500),
    LogisticRegression(multi_class="multinomial", solver="lbfgs", C=5),
    GaussianNB()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

C:\Users\pc\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\pc\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


KNeighborsClassifier
****Results****
Accuracy: 81.2688%
Log Loss: 2.9959835066469274
DecisionTreeClassifier
****Results****
Accuracy: 78.7786%
Log Loss: 0.5338870055824441
RandomForestClassifier
****Results****
Accuracy: 70.2805%
Log Loss: 0.7392988833506068
MLPClassifier
****Results****
Accuracy: 78.3184%
Log Loss: 0.5354913675477749
LogisticRegression
****Results****
Accuracy: 72.5560%
Log Loss: 0.6349777443197835
GaussianNB
****Results****
Accuracy: 66.9998%
Log Loss: 1.0607688930940729


In [ ]:
## El mejor modelo de clasificacion fue el KNN

In [19]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
for train_index, test_index in kf.split(X, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(3).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.653908  0.749110  0.698279   36239.0   41515.0  0.870807
1             0.708339  0.697401  0.702828   63728.0   62744.0  0.791592
2             0.579984  0.533222  0.555621   53233.0   48941.0  0.711137
avg / total   0.650863  0.652585  0.650601  153200.0  153200.0  0.797000,              precision    recall  f1-score   support      pred       AUC
0             0.658675  0.664836  0.661741   36239.0   36578.0  0.836203
1             0.696064  0.706048  0.701020   63728.0   64642.0  0.786963
2             0.560726  0.547528  0.554049   53232.0   51979.0  0.703131
avg / total   0.640194  0.641218  0.640661  153199.0  153199.0  0.785801,              precision    recall  f1-score   support      pred       AUC
0             0.660252  0.707277  0.682956   36239.0   38820.0  0.851498
1             0.702358  0.705290  0.703821   63727.0   63993.0  0.789693
2             0.567728  0.537365  0.552129   532